In [1]:
import cv2 as cv
import numpy as np
import time
from skimage.feature import local_binary_pattern
from scipy.stats import mode

In [2]:
image_train=np.load("E:/1-Collage/3-Senior-2/CV/Project/data_set/image_train.npy")
label_train=np.load("E:/1-Collage/3-Senior-2/CV/Project/data_set/label_train.npy")
image_test=np.load("E:/1-Collage/3-Senior-2/CV/Project/data_set/image_test.npy")
label_test=np.load("E:/1-Collage/3-Senior-2/CV/Project/data_set/label_test.npy")

In [3]:
radius = 1
n_points = 8 * radius
method = 'uniform'

# Shuffle the training data and labels
indices = np.arange(len(image_train))
np.random.shuffle(indices)
image_train = image_train[indices]
label_train = label_train[indices]

# Shuffle the test data and labels
indices = np.arange(len(image_test))
np.random.shuffle(indices)
image_test = image_test[indices]
label_test = label_test[indices]


LBP_train = [
    local_binary_pattern(cv.cvtColor(img, cv.COLOR_RGB2GRAY), n_points, radius, method)
    for img in image_train
]
LBP_test = [
    local_binary_pattern(cv.cvtColor(img, cv.COLOR_RGB2GRAY), n_points, radius, method)
    for img in image_test
]

In [4]:
n_bins = n_points + 2
LBP_train_histogram = np.array([
    cv.calcHist([lbp.astype('float32')], [0], None, [n_bins], [0, n_bins]).flatten()
    for lbp in LBP_train
])
LBP_test_histogram = np.array([
    cv.calcHist([lbp.astype('float32')], [0], None, [n_bins], [0, n_bins]).flatten()
    for lbp in LBP_test
])

LBP_all=np.concatenate((LBP_train_histogram,LBP_test_histogram),axis=0)
mean = np.mean(LBP_all, axis=0)
std = np.std(LBP_all, axis=0)

LBP_train_histogram = (LBP_train_histogram - mean) / std
LBP_test_histogram = (LBP_test_histogram - mean) / std


y_train = label_train.flatten()
y_test = label_test.flatten()

In [12]:
def knn_predict(X_train, y_train, X_test, k=3):
    predictions = []
    for x_test in X_test:
        # Compute distances to all training points
        distances = np.linalg.norm(X_train - x_test, axis=1)

        # Find the k nearest neighbors
        k_indices = np.argsort(distances)[:k]
        k_labels = y_train[k_indices]

        # Determine the most common label
        unique_labels, counts = np.unique(k_labels, return_counts=True)
        most_common = unique_labels[np.argmax(counts)]
        
        predictions.append(most_common)

    return np.array(predictions)



y_pred = knn_predict(LBP_train_histogram, y_train, LBP_test_histogram, k=3)  # Use a subset of test data for speed

accuracy = np.mean(y_pred == y_test)  # Match the subset size
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 60.34%


In [ ]:
point=12250
print(f"Prediction: {y_pred[point]}")
print(f"Prediction: {y_test[point]}")

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from skimage.feature import local_binary_pattern

# Train k-NN
knn = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
knn.fit(LBP_train_histogram, y_train)

# Predict and evaluate
y_pred = knn.predict(LBP_test_histogram)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 60.34%
